<a href="https://colab.research.google.com/github/sexymonster/Dacon/blob/main/bok_choy/bok_choy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 인트로

## 1-1. 내용 설명

[주제]
환경 데이터로부터 청경채의 일별 잎면적 증감률 예측하는 AI 알고리즘 개발

[설명]
각 청경채 케이스의 생육 기간동안 1분 간격으로 수집된 환경 데이터로부터 일별 잎면적 증감률을 예측

[링크]
https://dacon.io/competitions/official/235961/overview/description

## 1-2. 패키지 로드

In [ ]:
from google.colab import drive

import pandas as pd
import os
import glob

## 1-3. 데이터 로드

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/MyDrive/bok_choy/"

train_input_list = sorted(glob.glob(path+"train_input/*.csv"))
train_target_list = sorted(glob.glob(path+"train_target/*.csv"))
test_input_list = sorted(glob.glob(path+"test_input/*.csv"))
test_target_list = sorted(glob.glob(path+"test_target/*.csv"))

# 데이터 전처리

## 2-1. 데이터 확인

In [ ]:
train_input_data = []
for i in train_input_list:
  data = pd.read_csv(i)
  train_input_data.append(data)

In [ ]:
train_input_data[0]

,시간,내부온도관측치,내부습도관측치,CO2관측치,EC관측치,외부온도관측치,외부습도관측치,펌프상태,펌프작동남은시간,최근분무량,...,카메라상태,냉방온도,난방온도,기준온도,난방부하,냉방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량
0,2021-02-17 00:00:00,24.799999,40.500000,361.0,0.059069,20.299999,33.200001,0.0,0.0,0.0,...,4.5,0.0,0.0,363.554,16.548,37.596000,363.554,309.41,16.548,37.596
1,2021-02-17 00:01:00,20.500000,44.099998,355.0,0.910134,19.700001,10.700000,0.0,0.0,0.0,...,0.0,20.0,18.0,19.000,0.000,2.500000,0.000,0.00,0.000,0.000
2,2021-02-17 00:02:00,20.600000,44.299999,360.0,0.910134,19.799999,10.600000,0.0,0.0,0.0,...,0.0,20.0,18.0,19.000,0.000,3.000002,0.000,0.00,0.000,0.000
3,2021-02-17 00:03:00,20.600000,44.500000,359.0,0.908579,19.799999,10.400000,0.0,0.0,0.0,...,0.0,20.0,18.0,19.000,0.000,3.000002,0.000,0.00,0.000,0.000
4,2021-02-17 00:04:00,20.600000,44.500000,357.0,0.910134,19.799999,10.300000,0.0,0.0,0.0,...,0.0,20.0,18.0,19.000,0.000,3.000002,0.000,0.00,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41755,2021-03-17 23:55:00,20.400000,78.000000,470.0,0.313223,20.400000,31.500000,0.0,0.0,769.0,...,0.0,20.0,18.0,19.000,0.000,1.999998,0.000,0.00,0.000,0.000
41756,2021-03-17 23:56:00,20.400000,78.199997,471.0,0.314777,20.500000,31.400000,0.0,0.0,769.0,...,0.0,20.0,18.0,19.000,0.000,1.999998,0.000,0.00,0.000,0.000
41757,2021-03-17 23:57:00,20.500000,79.199997,468.0,0.315554,20.600000,31.299999,0.0,0.0,769.0,...,0.0,20.0,18.0,19.000,0.000,1.999998,0.000,0.00,0.000,0.000
41758,2021-03-17 23:58:00,20.600000,79.599998,471.0,0.316332,20.799999,31.200001,0.0,0.0,769.0,...,0.0,20.0,18.0,19.000,0.000,2.500000,0.000,0.00,0.000,0.000


## 2-2. 데이터셋 커스텀

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, input_paths, target_paths, infer_mode):
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.infer_mode = infer_mode
        
        self.data_list = []
        self.label_list = []
        print('Data Pre-processing..')
        for input_path, target_path in tqdm(zip(self.input_paths, self.target_paths)):
            input_df = pd.read_csv(input_path)
            target_df = pd.read_csv(target_path)
            
            input_df = input_df.drop(columns=['시간'])
            input_df = input_df.fillna(0)
            
            input_length = int(len(input_df)/1440)
            target_length = int(len(target_df))
            
            for idx in range(target_length):
                time_series = input_df[1440*idx:1440*(idx+1)].values
                self.data_list.append(torch.Tensor(time_series))
            for label in target_df["rate"]:
                self.label_list.append(label)
        print('Done.')
              
    def __getitem__(self, index):
        data = self.data_list[index]
        label = self.label_list[index]
        if self.infer_mode == False:
            return data, label
        else:
            return data
        
    def __len__(self):
        return len(self.data_list)